In [2]:
import pandas as pd
import os
from datetime import datetime, timedelta

# ==========================================
# 설정 (Settings)
# ==========================================
symbol = "BTCUSDT"
date_str = "2025-03-01"
top_n = 50
window = 60 # 검증할 윈도우 크기 (60분)

# 파일 경로
x_path = f"data/x/{date_str}_x_top{top_n}.h5"
raw_dir = "data/1m_raw_data"

# ==========================================
# 1. 데이터 로드 및 병합
# ==========================================
print(f"[INFO] {symbol} 데이터 추출 중...")

# (1) X 데이터 (Python이 계산한 Feature) 로드
if not os.path.exists(x_path):
 raise FileNotFoundError(f"X 파일 없음: {x_path}")

df_x = pd.read_hdf(x_path, key="x")
df_x = df_x[df_x["symbol"] == symbol].copy()

# Feature 컬럼만 선택 (중립화된 _neut 제외, 원본 Feature 값만 확인)
feat_cols = [c for c in df_x.columns if c.startswith(f"x_{window}m_") and not c.endswith("_neut")]
# 비교를 위해 필요한 시간, 심볼 정보
df_x_subset = df_x[["start_time_ms", "start_time_utc"] + feat_cols].copy()

# (2) Raw 데이터 (OHLCV 원본) 로드
# 윈도우 계산을 위해 전날 데이터도 필요
target_date = datetime.strptime(date_str, "%Y-%m-%d").date()
prev_date_str = (target_date - timedelta(days=1)).strftime("%Y-%m-%d")
raw_paths = [os.path.join(raw_dir, f"{d}.h5") for d in [prev_date_str, date_str]]

raw_list = []
for p in raw_paths:
 if os.path.exists(p):
 raw_list.append(pd.read_hdf(p))

if not raw_list:
 raise ValueError("원본 데이터 파일이 없습니다.")

df_raw = pd.concat(raw_list, ignore_index=True)
df_raw = df_raw[df_raw["symbol"] == symbol].copy()
df_raw = df_raw.sort_values("start_time_ms")

# (3) 두 데이터 병합 (Time 기준)
# Raw 데이터에 Feature 데이터를 붙임
merged = pd.merge(df_raw, df_x_subset, on=["start_time_ms", "start_time_utc"], how="left")

# 엑셀에서 보기 편하게 타겟 날짜 데이터만 남기되, 
# 엑셀 수식 검증을 위해 앞쪽 데이터(전날 23시~)도 일부 포함하여 저장
cutoff_time = df_x_subset["start_time_ms"].min() - (window * 60 * 1000 * 2) # 넉넉하게 윈도우 2배 전부터
merged_save = merged[merged["start_time_ms"] >= cutoff_time].copy()

# ==========================================
# 2. 엑셀로 내보내기
# ==========================================
output_file = f"{symbol}_verification_window{window}.xlsx"
merged_save.to_excel(output_file, index=False)

print(f"[DONE] 파일 생성 완료: {output_file}") # feature가 올바르게 계산되었는 지 확인하기 위해 excel 수식을 이용해 검증하였음